# Task 6.2 from *SemEval*
## Imports

In [1]:
import torch

from src.dataprocessing import preprocesser, postprocesser
from src.model import model

import json

## Input Data Preprocessing

In [2]:
# preprocesser.augment_data("propaganda_detection/datasets/train-articles","propaganda_detection/datasets/train-labels-task2-technique-classification", "out/augmented_data")

In [3]:
train_file_path = "SEMEVAL-2021-task6-corpus/data/training_set_task2.txt"
with open(train_file_path, "r") as file:
    train_data = json.load(file)

train_aug_file_path = "out/augmented_data.json"
with open(train_aug_file_path, "r") as file:
    train_aug_data = json.load(file)

dev_file_path = "SEMEVAL-2021-task6-corpus/data/dev_set_task2.txt"
with open(dev_file_path, "r") as file:
    dev_data = json.load(file)

test_file_path = "SEMEVAL-2021-task6-corpus/data/test_set_task2.txt"
with open(dev_file_path, "r") as file:
    test_data = json.load(file)

input_tdata, output_tdata = preprocesser.preprocess_data(train_data)
input_taugdata, output_taugdata = preprocesser.preprocess_data(train_aug_data)
input_vdata, output_vdata = preprocesser.preprocess_data(dev_data)

# Data Augmentation ö
input_tdata.extend(input_taugdata)
output_tdata.extend(output_taugdata)

In [4]:
model.train(input_tdata, output_tdata, input_vdata, output_vdata)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be ab

## Results & Post-Processing

In [5]:
model = torch.load('models/MEMbErt.pth')
predictions = []

with torch.no_grad():
    for meme in input_vdata:
        model.eval()
        predictions.append(model(' '.join(meme)).cpu().numpy())

In [6]:
predictions[50]

array([[0.37854278, 0.40294778, 0.5278869 , 0.44029826, 0.41455758,
        0.4337486 , 0.49179694, 0.16653179, 0.44977438, 0.1556831 ,
        0.47526997, 0.14861926, 0.34056258, 0.37895724, 0.33433878,
        0.38427767, 0.35314292, 0.3330341 , 0.3585393 , 0.3791072 ],
       [0.29778934, 0.29213557, 0.38569048, 0.3754421 , 0.34032017,
        0.3138022 , 0.41564265, 0.05766053, 0.36052066, 0.03363029,
        0.25664377, 0.03557968, 0.14964865, 0.1777665 , 0.35076118,
        0.26844695, 0.16015072, 0.11783781, 0.16764832, 0.17746285],
       [0.34477282, 0.40457052, 0.62894225, 0.45158944, 0.40990654,
        0.4076273 , 0.30397347, 0.09522396, 0.40554243, 0.06266552,
        0.37417606, 0.06583284, 0.2720749 , 0.3363626 , 0.32857156,
        0.32680684, 0.30143148, 0.2725619 , 0.31812945, 0.34321213],
       [0.36765483, 0.43329427, 0.496691  , 0.39634284, 0.38629246,
        0.4186626 , 0.3616237 , 0.153953  , 0.355605  , 0.12302508,
        0.39194304, 0.11019623, 0.3270484 , 0

In [9]:
output = postprocesser.postprocess_data(dev_data, input_vdata, predictions, model.tokenizer, threshold=0.6)
with open("out/out.txt", "w") as f:
    json.dump(output, f)

## Evaluation
For evaluation, type:

`python3 SEMEVAL-2021-task6-corpus/scorer/task2/task-2-semeval21_scorer.py -s out/out.txt -r SEMEVAL-2021-task6-corpus/data/dev_set_task2.txt -p SEMEVAL-2021-task6-corpus/techniques_list_task1-2.txt
`